## Q. Titanic 생존자 예측 모델 성능을 개선하고자 아래와 같이 교차 검증 및 하이퍼 파라미터 튜닝을 수행하세요.
 - KFold 교차검증(랜덤포레스트 적용)
 - cross_val_score를 활용한 교차 검증(랜덤 포레스트 적용)
 - GridSearchCV를 활용한 교차 검증과 최적 하이퍼파라미터 수행하고 성능을 평가
   - RandomForestClassifier의 최적 하이퍼 파라미터를 찾고 예측 성능 측정 parameters = {'n_estimators':[50,100,200],'max_depth':[2,3,5],'min_samples_leaf':[1,5,8]}
   - LogisticRegression의 최적 하이퍼 파라미터를 찾고 예측 성능 측정 parameters = {'penalty':['l2','l1'],'C':[0.01,0.1,1,1,5,10]}
 
#### RF 파라미터
 - n_estimators : (나무의 개수)랜덤 포레스트에서 결정 트리의 개수를 지정. 디폴트는 10개
    - 많이 설정할 수록 좋은 성능을 기대할 수  있지만 학습 수행시간이 오래 걸림
 - max_features : 결정 트리에 사용된 max_features 파라미터와 같음. sqrt(전체 피처 개수)만큼 참조. 피터가 16개라면 분할 위해 4개 참조
 
#### LR 파라미터
    - (과재적합을 피하기 위해 회귀계수를 조정해야하는데, 조정하는 방식을 말한다.)
 - L1 : 불필요한 회귀계수를 급격하게 감소시켜 0으로 만들고 제거
 - L2 : 회귀 계수의 크기를 감소시킴
 - C : 규제 강도를 조절하는 alpha 값의 역수. 즉 C값이 작을 수록 규제 강도가 큼

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [13]:
# 독립변수, 종속변수 분리
t_df = pd.read_pickle('t_df.pkl')

y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

# 학습용 평가용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

# 분류기 객체 생성
# dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
# lr_clf = LogisticRegression(random_state=11)

from sklearn.model_selection import KFold

# KFold 교차 검증 수행()
# K개의 데이터 폴드 세트를 만들어서 K번 만큼 각 폴드 세트에
# 학습과 검증 평가를 반복, K가 5이면 5번 평가를 평균한 결과로 예측 성능 검증
# 예측 성능 평가
def exec_kfold(clf, folds=5):      # clf는 클래시파이어, folds는 k의 개수
    kfold = KFold(n_splits=folds)  # n_splits = folds
    scores = []                    # 만들어서 저장하는 공간( foltset 설정하는 단계 )
    
    for iter_count, (train_index, test_index) \
    in enumerate(kfold.split(X_df)):            # enumerate 해서 독립변수 쪽의 5개를 하나씩 작업해준다.
        X_train, X_test = X_df.values[train_index], X_df.values[test_index]
        y_train, y_test = y_df.values[train_index], y_df.values[test_index]
        
        clf.fit(X_train, y_train)                    # 반복학습 ( 이 작업들을 각 fold마다 진행 )
        predictions = clf.predict(X_test)            # 예측값 삽입 
        accuracy = accuracy_score(y_test, predictions) # 각 폴드마다 한 것을 append 해준다.
        scores.append(accuracy)                      # 5개에 대한 정확도를 출력
        print('교차 검증 {0}정확도: {1:.4f}'.format(iter_count, accuracy))
        # print('검증 세트 인덱스: {1}'.format(iter_count, test_index))
        
    mean_score = np.mean(scores)                   # mean_score로 평균을 구해준다.(5개 결과를 저장한 scores를 호출하여)
    print('평균 정확도: {0: .4f}'.format(mean_score))
        
exec_kfold(rf_clf, folds=5)       # 5로 변경하면 5개가 출력된다.

교차 검증 0정확도: 0.7710
교차 검증 1정확도: 0.8473
교차 검증 2정확도: 0.7405
교차 검증 3정확도: 0.6947
교차 검증 4정확도: 0.7816
평균 정확도:  0.7670


In [16]:
# cross_val_scores
# KFold의 일련의 과정을 한꺼번에 수행해주는 API

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_clf, X_df, y_df, cv=10)        # cv=5는 교차검증을 얼마나 할 것인가, 숫자를 올리면 KFold와 비슷해진다.
for iter_count, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))
    
print('평균 정확도:{0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.5573
교차 검증 1 정확도: 0.8397
교차 검증 2 정확도: 0.8092
교차 검증 3 정확도: 0.7939
교차 검증 4 정확도: 0.7710
교차 검증 5 정확도: 0.8397
교차 검증 6 정확도: 0.7099
교차 검증 7 정확도: 0.6183
교차 검증 8 정확도: 0.6565
교차 검증 9 정확도: 0.6923
평균 정확도:0.7288


In [21]:
t_df = pd.read_pickle('t_df.pkl')

y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

# 학습용 평가용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

# 분류기 객체 생성
dt_clf = DecisionTreeClassifier()

# GridSearchCV : 파라미터를 통해 성능을 튜닝. (타이타닉 생존률을 해결하기 위해 필요한 작업)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

parameters = {'n_estimators':[50,100,200],'max_depth':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_dclf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

display(grid_dclf)
grid_dclf.fit(X_train, y_train) 

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_) 

print('GridSearchCV 최고 정확도:', grid_dclf.best_score_) 

best_dclf = grid_dclf.best_estimator_
display(best_dclf)
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print('GSCV 예측 정확도 : ', accuracy) 

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=11,
                                  

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 200}
GridSearchCV 최고 정확도: 0.8042105263157895


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=11, verbose=0,
                       warm_start=False)

GSCV 예측 정확도 :  0.7977099236641222
